In [1]:
import pandas as pd

In [2]:
def extract_labs_products_list(
    prep_transactor_datacube: pd.DataFrame,
    prep_evident_datacube: pd.DataFrame,
    prep_labtrac_orders_data: pd.DataFrame,
    prep_leca_orders_data: pd.DataFrame,
    prep_ashford: pd.DataFrame,
):
    """
    Outputs products list directly extracted from preprocessed data across all ALS labs using Transactor, Evident and
    Labtrac. Note that this initial products list will contain many duplicated entries. Note that this initial products
    list only extracts unique product IDs at the lab-level, not across all labs. There may be some duplicated entries
    within the group of products extracted for ea ch lab due to lab-side entry errors.
    :param prep_transactor_datacube: DataFrame containing combined Transactor data cube across all labs using
    Transactor.
    :param prep_evident_datacube: DataFrame containing combined Evident data cube across all labs using Evident.
    :param prep_labtrac_orders_data: DataFrame containing combined Labtrac orders data across all labs using Labtrac.
    :param prep_leca_orders_data: DataFrame containing combined Leca orders data.
    : param prep_ashford: DataFrame containing combined Ashford datacube across all labs.
    :return prep_all_labs_products_list: DataFrame containing apparent unique products with original IDs from lab-level
    data.
    """
    # Get Transactor products list
    prep_transactor_products_list = (
        prep_transactor_datacube.groupby(
            ["product_code", "product_description", "als_lab", "system_source"], as_index=False
        )["net_sales"]
        .sum()        
    )

    # Get Evident products list
    prep_evident_products_list = (
        prep_evident_datacube.groupby(
            ["product_description", "als_lab", "system_source"],as_index=False
        )["net_sales"]
        .sum()        
    )
    prep_evident_products_list["product_code"] = "Unknown"
    prep_evident_products_list = prep_evident_products_list[
        ["product_code", "product_description", "als_lab", "system_source"]
    ]

    # Get Labtrac products list
    prep_labtrac_products_list = (
        prep_labtrac_orders_data.groupby(
            ["product_code", "product_description", "als_lab", "system_source"], as_index=False
        )["net_sales"]
        .sum()
    )

    # Get Leca products list
    prep_leca_products_list = (
        prep_leca_orders_data.groupby(
            ["product_code", "product_description", "als_lab", "system_source"], as_index=False
        )["net_sales"]
        .sum()
    )

    # Get Ashford products list
    prep_asford_products_list = (
        prep_ashford.groupby(
            ["product_code", "product_description", "als_lab", "system_source"], as_index=False
        )["net_sales"]
        .sum()
    )

    # Combine product lists across Transactor, Evident and Labtrac,
    prep_all_labs_products_list = pd.concat(
        [
            prep_transactor_products_list,
            prep_evident_products_list,
            prep_labtrac_products_list,
            prep_leca_products_list,
            prep_asford_products_list,
        ]
    )

    # Create combined products code column matching the lab product codes column in the ALS sku mapping file
    # Transactor, Labtrac, Leca and Ashford already have their own product codes in the data, so we will use those
    prep_all_labs_products_list.loc[
        (prep_all_labs_products_list["system_source"] == "Transactor")
        | (prep_all_labs_products_list["system_source"] == "Labtrac")
        | (prep_all_labs_products_list["system_source"] == "Custom"),
        "product_matching_code",
    ] = prep_all_labs_products_list.loc[
        (prep_all_labs_products_list["system_source"] == "Transactor")
        | (prep_all_labs_products_list["system_source"] == "Labtrac")
        | (prep_all_labs_products_list["system_source"] == "Custom"),
        "product_code",
    ]

    # Evident does not have its own product codes, so we are using the product descriptions
    prep_all_labs_products_list.loc[
        (prep_all_labs_products_list["system_source"] == "Evident"),
        "product_matching_code",
    ] = prep_all_labs_products_list.loc[
        (prep_all_labs_products_list["system_source"] == "Evident"),
        "product_description",
    ]

    # Sort the products lists table by lab for ease of viewing
    prep_all_labs_products_list = prep_all_labs_products_list.sort_values(
        by="als_lab", axis=0, ascending=False, ignore_index=True
    )

    prep_all_labs_products_list = prep_all_labs_products_list.rename(columns={"net_sales": "net_sales_prod_sku_map"})
    return prep_all_labs_products_list

In [3]:
def match_product_sku_data(
    prep_all_labs_products_list: pd.DataFrame,
    als_products_skus: pd.DataFrame,
    als_sku_lab_mapping: pd.DataFrame,
):
    """
    Match lab-level product codes and product data across all labs using Transactor, Evident and Labtrac to the ALS
    unified SKU and SKU product data.
    :param prep_all_labs_products_list: DataFrame containing apparent unique products with original IDs from lab-level
    data.
    :param als_products_skus: DataFrame of preprocessed ALS SKU codes and associated product information.
    :param als_sku_lab_mapping: DataFrame mapping ALS SKU codes to lab-level product codes.
    :return final_product_sku_mapping: DataFrame mapping ALS SKU codes to all extant lab-level product codes directly
    found in lab-level data, with associated ALS SKU code data.
    """
    for col in als_sku_lab_mapping.columns:
        als_sku_lab_mapping[col] = als_sku_lab_mapping[col].astype(str)
        als_sku_lab_mapping[col] = als_sku_lab_mapping[col].str.strip()
    
    for col in prep_all_labs_products_list:
        prep_all_labs_products_list[col] = prep_all_labs_products_list[col].astype(str)
        prep_all_labs_products_list[col] = prep_all_labs_products_list[col].str.strip()
    
    for col in als_products_skus:
        als_products_skus[col] = als_products_skus[col].astype(str)
        als_products_skus[col] = als_products_skus[col].str.strip()

    # Ensure that all ALS lab names are correct

    # Create output table by joining lab product codes to ALS SKU codes
    final_product_sku_mapping = prep_all_labs_products_list.merge(
        als_sku_lab_mapping,
        left_on=["als_lab", "product_matching_code"],
        right_on=["als_lab_name", "lab_code"],
        how="left",
    )

    # Join ALS SKU data to output table
    final_product_sku_mapping = final_product_sku_mapping.merge(
        als_products_skus, on="als_sku_code", how="left"
    )

    final_product_sku_mapping.loc[final_product_sku_mapping["als_product_type"] == "Digital", "als_product_type"] = "% Digital"
    final_product_sku_mapping.loc[
        final_product_sku_mapping["als_product_type"] == "Analogue", "als_product_type"] = "% Analogue"

    for code in ['MO0510801G', 'MO0304701B', 'MISC001AD', 'PR0208702G', 'PR0208702B', 'PR0208702S', 'MISC001Ad', 'OR0210029B', 'OR0210029G', 'OR0206729G', 'PR0610615S', 'PR0610615B', 'PR0610615G', 'AD0511399Z', 'Denture Misc', 'g', 'DISCOU|NT', 'DLIVERY', 'Misc', 'Implants MISC', 'MISC', 'Delivery', 'Models', 'DISCOUNT', 'Unknown']:
        final_product_sku_mapping.loc[final_product_sku_mapping["als_sku_code"] == code, "als_sku_code"] = "MISC0001AD"
        final_product_sku_mapping.loc[final_product_sku_mapping["als_sku_code"] == code, ["als_product_category", "als_product_sub_category", "als_product_name", "als_product_type"]] = "Miscellaneous"

    return final_product_sku_mapping


In [4]:
def join_labs_data_als_sku(
    nhs_private_tagged_transactor_datacube,
    nhs_private_tagged_evident_datacube,
    nhs_private_tagged_labtrac_orders_data,
    nhs_private_tagged_leca_orders_data,
    nhs_private_tagged_ashford_orders_data,
    final_product_sku_mapping,
):
    """
    Join the preprocessed labs datacube/orders data for each system (Transactor, Evident and Labtrac) that has been
    tagged at the row-level by whether sales were to NHS, private or ALS lab customers with ALS SKU codes and
    associated data.
    :param nhs_private_tagged_transactor_datacube: DataFrame containing lab-level preprocessed data with sales tagged
    to NHS/private/ALS lab customers for Transactor.
    :param nhs_private_tagged_evident_datacube: DataFrame containing lab-level preprocessed data with sales tagged
    to NHS/private/ALS lab customers for Evident.
    :param nhs_private_tagged_labtrac_orders_data: DataFrame containing lab-level preprocessed data with sales tagged
    to NHS/private/ALS lab customers for Labtrac.
    :param nhs_private_tagged_leca_orders_data: DataFrame containing lab-level preprocessed data with sales tagged
    to NHS/private/ALS lab customers for Leca.
    :param nhs_private_tagged_ashford_orders_data: DataFrame containing lab-level preprocessed data with sales tagged
    to NHS/private/ALS lab customers for Ashford.
    :param final_product_sku_mapping: DataFrame containing apparent unique products with original IDs from lab-level
    data.
    :return als_sku_transactor_datacube, als_sku_evident_datacube, als_sku_labtrac_orders_data: DataFrames with labs
    data that has been tagged at the row-level by whether sales were to NHS, private or ALS lab customers joined to ALS
    SKU codes and associated data, for each of Transactor, Evident and Labtrac (in that order).
    """
    # Create output tables for manipulation and joining
    als_sku_transactor_datacube = nhs_private_tagged_transactor_datacube.copy(deep=True)
    als_sku_evident_datacube = nhs_private_tagged_evident_datacube.copy(deep=True)
    als_sku_labtrac_orders_data = nhs_private_tagged_labtrac_orders_data.copy(deep=True)
    als_sku_leca_orders_data = nhs_private_tagged_leca_orders_data.copy(deep=True)
    als_sku_ashford_orders_data = nhs_private_tagged_ashford_orders_data.copy(deep=True)

    # Drop extraneous columns from the product-code-ALS-SKU-mapping table
    final_product_sku_mapping_shortened = final_product_sku_mapping.drop(
        columns=[
            "product_code",
            "system_source",
            "raw_lab_name",
            "als_lab_name",
            "lab_code",
        ]
    ).copy(deep=True)

    final_product_sku_mapping_shortened[
        "product_matching_code"
    ] = final_product_sku_mapping_shortened["product_matching_code"].astype(str)

    final_product_sku_mapping_shortened = final_product_sku_mapping_shortened.iloc[::-1]

    digital_products = final_product_sku_mapping_shortened.loc[final_product_sku_mapping_shortened["als_product_type"] == "% Digital "].reset_index(drop=True)

    final_product_sku_mapping_shortened = (
        final_product_sku_mapping_shortened.drop_duplicates(
            subset=["product_description", "als_lab", "product_matching_code"], keep="first"
        )
    )

    final_product_sku_mapping_shortened = pd.concat([digital_products, final_product_sku_mapping_shortened]).drop_duplicates(subset=["product_description", "als_lab", "product_matching_code"], keep="first")

    # Join labs/system data tables to ALS SKU data
    for df, code_col, system_source in [
        (als_sku_transactor_datacube, "product_code", "Transactor"),
        (als_sku_evident_datacube, "product_description", "Evident"),
        (als_sku_labtrac_orders_data, "product_code", "Labtrac"),
        (als_sku_leca_orders_data, "product_code", "Leca"),
        (als_sku_ashford_orders_data, "product_code", "Ashford"),
    ]:
        if code_col == "product_description":
            df = df.merge(
                final_product_sku_mapping_shortened,
                left_on=[code_col, "als_lab"],
                right_on=["product_matching_code", "als_lab"],
                how="left",
            )
        elif code_col == "product_code":
            df = df.merge(
                final_product_sku_mapping_shortened,
                left_on=[code_col, "product_description", "als_lab"],
                right_on=["product_matching_code", "product_description", "als_lab"],
                how="left",
            )
        if system_source == "Transactor":
            als_sku_transactor_datacube = df.drop(columns=["product_matching_code"])
        if system_source == "Evident":
            als_sku_evident_datacube = df.drop(
                columns=["product_matching_code", "product_description_y"]
            ).rename(columns={"product_description_x": "product_description"})
        if system_source == "Labtrac":
            als_sku_labtrac_orders_data = df.drop(columns="product_matching_code")
        if system_source == "Leca":
            als_sku_leca_orders_data = df.drop(columns="product_matching_code")
        if system_source == "Ashford":
            als_sku_ashford_orders_data = df.drop(columns="product_matching_code")

    # Identify all lab-level products above revenue threshold that require manual categorisation to ALS SKU codes
    # Give the Evident datacube a redundant product code column
    als_sku_evident_datacube["product_code"] = "Unknown"

    # Iterate through all the system datacubes to identify the 5% revenue threshold for each lab
    products_to_categorise_list = []
    for df in [
        als_sku_transactor_datacube,
        als_sku_evident_datacube,
        als_sku_labtrac_orders_data,
        als_sku_leca_orders_data,
        als_sku_ashford_orders_data,
    ]:
        # Get sorted DataFrame of total net sales attached to each lab-level product over the entire multi-year period
        products = (
            df.loc[((df["als_sku_code"].isna()) | (df["als_sku_code"] == "MISC0001AD")), :]
            .groupby(["product_code", "product_description", "als_lab"])["net_sales"]
            .sum()
            .reset_index(drop=False)
            .sort_values("net_sales")
            .reset_index(drop=True)
        )

        # Get total net sales of each lab over the entire multi-year period
        labs = df.groupby("als_lab")["net_sales"].sum().reset_index()

        # Iterate through each lab to find the 5% revenue threshold and the products above that threshold to categorise
        for lab in labs["als_lab"].unique():
            # Find the 5% revenue threshold for the lab
            max_value = labs.loc[labs["als_lab"] == lab, "net_sales"].values[0] * 0.1
            # Find all the uncategorise products for a given lab
            lab_products = products.loc[products["als_lab"] == lab, :].reset_index(
                drop=True
            )
            # Get column of the cumulative sum of the sales from the uncategorised products
            lab_products["cumsum"] = lab_products["net_sales"].cumsum()

            # Identify the specific product at which point the cumulative uncategorised products sales cumulative sum
            # exceeds the 5% revenue threshold for the lab
            lab_products["max_value_bool"] = False
            lab_products.loc[
                lab_products["cumsum"] > max_value, "max_value_bool"
            ] = True

            # Get DataFrame of all the products exceeding the 5% revenue threshold that must be
            lab_products_to_categorise = lab_products.loc[
                lab_products["max_value_bool"] == True, :
            ]
            products_to_categorise_list.append(lab_products_to_categorise)
    products_to_categorise = pd.concat(products_to_categorise_list).reset_index(
        drop=True
    )

    # als_sku_evident_datacube = als_sku_evident_datacube.drop(columns="product_code")

    datacube = pd.concat(
        [
            als_sku_transactor_datacube[["product_code", "product_description","als_lab", "net_sales", "als_sku_code"]],
            als_sku_evident_datacube[["product_code", "product_description","als_lab", "net_sales", "als_sku_code"]],
            als_sku_labtrac_orders_data[["product_code", "product_description","als_lab", "net_sales", "als_sku_code"]],
            als_sku_leca_orders_data[["product_code", "product_description","als_lab", "net_sales", "als_sku_code"]],
            als_sku_ashford_orders_data[["product_code", "product_description","als_lab", "net_sales", "als_sku_code"]],
        ]
    )

    products = (
        datacube.loc[((datacube["als_sku_code"].isna()) | (datacube["als_sku_code"] == "MISC0001AD")), :]
        .groupby(["product_code", "product_description", "als_lab"])["net_sales"]
        .sum()
        .reset_index(drop=False)
        .sort_values("net_sales")
        .reset_index(drop=True)
    )

    # Find the 5% revenue threshold for the lab
    max_value = datacube["net_sales"].sum() * 0.1
    # Get column of the cumulative sum of the sales from the uncategorised products
    products["cumsum"] = products["net_sales"].cumsum()

    # Identify the specific product at which point the cumulative uncategorised products sales cumulative sum
    # exceeds the 5% revenue threshold for the lab
    products["max_value_bool"] = False
    products.loc[
        products["cumsum"] > max_value, "max_value_bool"
    ] = True

    # Get DataFrame of all the products exceeding the 5% revenue threshold that must be
    to_categorise = products.loc[
                                 products["max_value_bool"] == True, :
                                 ]

    to_categorise = to_categorise.merge(products_to_categorise, on=["product_code", "product_description", "als_lab"], how="left")

    to_categorise = to_categorise.loc[to_categorise["net_sales_y"].isna()].sort_values(by="net_sales_x", ascending=False)

    return (
        als_sku_transactor_datacube,
        als_sku_evident_datacube,
        als_sku_labtrac_orders_data,
        als_sku_leca_orders_data,
        als_sku_ashford_orders_data,
        products_to_categorise,
    )


In [5]:
output_folder = "data\processed_product_mapping"

In [6]:
folder_path = "data/pre_processed_combined/sales"
prep_transactor_datacube = pd.read_csv(f"{folder_path}/combined_transactor.csv")
prep_evident_datacube = pd.read_csv(f"{folder_path}/combined_Densign.csv") 
prep_labtrac_orders_data = pd.read_csv(f"{folder_path}/combined_labtrac.csv")
prep_leca_orders_data = pd.read_csv(f"{folder_path}/combined_leca.csv")
prep_ashford = pd.read_csv(f"{folder_path}/combined_Ashford.csv")

prep_all_labs_products_list = extract_labs_products_list(prep_transactor_datacube,prep_evident_datacube,prep_labtrac_orders_data,prep_leca_orders_data,prep_ashford)
prep_all_labs_products_list.to_csv(f"{output_folder}/prep_all_labs_products_list.csv", index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_30136\3674626280.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  prep_evident_datacube = pd.read_csv(f"{folder_path}/combined_Densign.csv")
C:\Users\Admin\AppData\Local\Temp\ipykernel_30136\3674626280.py:4: DtypeWarning: Columns (10,14,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  prep_labtrac_orders_data = pd.read_csv(f"{folder_path}/combined_labtrac.csv")
C:\Users\Admin\AppData\Local\Temp\ipykernel_30136\3674626280.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  prep_ashford = pd.read_csv(f"{folder_path}/combined_Ashford.csv")


In [7]:
folder_path = "data/03_intermediate/sku_mappings"
als_products_skus = pd.read_csv(f"{folder_path}/als_product_skus.csv")
als_sku_lab_mapping = pd.read_csv(f"{folder_path}/als_sku_lab_mapping.csv")

In [8]:
final_product_sku_mapping = match_product_sku_data(prep_all_labs_products_list,als_products_skus,als_sku_lab_mapping)
final_product_sku_mapping.to_csv(f"{output_folder}/final_product_sku_mapping.csv")

In [11]:
final_product_sku_mapping

,product_code,product_description,als_lab,system_source,net_sales_prod_sku_map,product_matching_code,als_sku_code,lab_code,raw_lab_name,als_lab_name,als_product_category,als_product_sub_category,als_product_name,als_product_material,als_product_standard,als_product_type
0,C203,E Max Crown,Woodford,Labtrac,59236.7,C203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C165,Vita Shade Guide,Woodford,Labtrac,400.0,C165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C156,Dovetail Slot,Woodford,Labtrac,55.4,C156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C157,Fitting Of Attachment,Woodford,Labtrac,474.29999999999995,C157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C159,Pink Porcelain/Unit,Woodford,Labtrac,100.0,C159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23334,CREATSRB,Createch Screw Retained Bridge,APlus,Labtrac,132161.9,CREATSRB,CB0102608G,CREATSRB,A Plus,APlus,Crown & Bridge,Bridge,Bridge,Composite,Gold,% Analogue
23335,CRI,Cold Cure Reline,APlus,Labtrac,0.0,CRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23336,R/RSP,Rest/Rest Seat,APlus,Labtrac,39.599999999999994,R/RSP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23337,R/RSI,Rest/Rest Seat,APlus,Labtrac,62.0,R/RSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
folder_path = "data/03_intermediate/nhs_private_tagged_data"
nhs_private_tagged_transactor_datacube = pd.read_csv(f"{folder_path}/nhs_private_tagged_transactor_datacube.csv")
nhs_private_tagged_evident_datacube = pd.read_csv(f"{folder_path}/nhs_private_tagged_evident_datacube.csv")
nhs_private_tagged_labtrac_orders_data = pd.read_csv(f"{folder_path}/nhs_private_tagged_labtrac_orders_data.csv")
nhs_private_tagged_leca_orders_data = pd.read_csv(f"{folder_path}/nhs_private_tagged_leca_orders_data.csv")
nhs_private_tagged_ashford_orders_data = pd.read_csv(f"{folder_path}/nhs_private_tagged_ashford_orders_data.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_30136\1698048799.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  nhs_private_tagged_evident_datacube = pd.read_csv(f"{folder_path}/nhs_private_tagged_evident_datacube.csv")
C:\Users\Admin\AppData\Local\Temp\ipykernel_30136\1698048799.py:4: DtypeWarning: Columns (10,12,15,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  nhs_private_tagged_labtrac_orders_data = pd.read_csv(f"{folder_path}/nhs_private_tagged_labtrac_orders_data.csv")
C:\Users\Admin\AppData\Local\Temp\ipykernel_30136\1698048799.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  nhs_private_tagged_ashford_orders_data = pd.read_csv(f"{folder_path}/nhs_private_tagged_ashford_orders_data.csv")


In [10]:
als_sku_transactor_datacube,als_sku_evident_datacube,als_sku_labtrac_orders_data,als_sku_leca_orders_data,als_sku_ashford_orders_data,products_to_categorise =join_labs_data_als_sku(nhs_private_tagged_transactor_datacube,nhs_private_tagged_evident_datacube,
                       nhs_private_tagged_labtrac_orders_data,nhs_private_tagged_leca_orders_data,
                       nhs_private_tagged_ashford_orders_data,final_product_sku_mapping)

als_sku_transactor_datacube.to_csv(f"{output_folder}/als_sku_transactor_datacube.csv")
als_sku_evident_datacube.to_csv(f"{output_folder}/als_sku_evident_datacube.csv")
als_sku_labtrac_orders_data.to_csv(f"{output_folder}/als_sku_labtrac_orders_data.csv")
als_sku_leca_orders_data.to_csv(f"{output_folder}/als_sku_leca_orders_data.csv")
als_sku_ashford_orders_data.to_csv(f"{output_folder}/als_sku_ashford_orders_data.csv")
products_to_categorise.to_csv(f"{output_folder}/product_to_categorise.csv")
